In [10]:
import scipy.io
import re
import numpy as np
from collections import Counter
import os

In [11]:
mat = scipy.io.loadmat('MR001A.mat')

mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Oct 22 13:59:07 2024',
 '__version__': '1.0',
 '__globals__': [],
 'matriz_resultante': array([[ -624.4,  -628.9,  -634.1, ...,  -535.2,  -533.2,  -531.5],
        [ -679.2,  -681.3,  -682.8, ...,  -644.7,  -642.6,  -641.2],
        [ -688.5,  -689.5,  -689.7, ...,  -623.5,  -621.2,  -619.3],
        ...,
        [ -678.2,  -680.5,  -681.9, ...,  -638.4,  -636.4,  -634.8],
        [ -659.9,  -660.7,  -660.9, ...,  -620.3,  -617.4,  -616.2],
        [ -602.7,  -622.8,  -672.3, ..., -1078.7,  -836.1,  -536.3]])}

In [12]:
dato = mat['matriz_resultante'].T
print(dato.shape)
dato

(96000, 32)


array([[ -624.4,  -679.2,  -688.5, ...,  -678.2,  -659.9,  -602.7],
       [ -628.9,  -681.3,  -689.5, ...,  -680.5,  -660.7,  -622.8],
       [ -634.1,  -682.8,  -689.7, ...,  -681.9,  -660.9,  -672.3],
       ...,
       [ -535.2,  -644.7,  -623.5, ...,  -638.4,  -620.3, -1078.7],
       [ -533.2,  -642.6,  -621.2, ...,  -636.4,  -617.4,  -836.1],
       [ -531.5,  -641.2,  -619.3, ...,  -634.8,  -616.2,  -536.3]])

In [14]:
train = ['MR003A.mat', 'MR009A.mat', 'MR010A.mat', 'MR012A.mat', 'MR014A.mat', 'MR015A.mat', 'MR017A.mat', 'MR023A.mat', 'MR024A.mat']
valid = ['MR011A.mat', 'MR004A.mat', 'MR013A.mat']
test = ['MR008A.mat', 'MR018A.mat', 'MR006A.mat']

In [ ]:
dir = './registros/Tabla/Correctos'

directory = os.fsencode(dir)

X_train = np.array([])
X_valid = np.array([])
X_test = np.array([])
c1 = 0
c2 = 0
c3 = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".mat"):
        mat_aux = scipy.io.loadmat(dir + '/' + filename)['matriz_resultante'].T

        #bloque = 500
        #mascara = np.arange(len(mat_aux)) % (2 * bloque) < bloque
        #mat_aux = mat_aux[mascara]

        if filename in train:
            if c1 == 0:
                X_train = mat_aux
            else:
                X_train = np.concatenate((X_train, mat_aux), axis=0)
            c1 += 1
        elif filename in valid:
            if c2 == 0:
                X_valid = mat_aux
            else:
                X_valid = np.concatenate((X_valid, mat_aux), axis=0)
            c2 += 1
        elif filename in test:
            if c3 == 0:
                X_test = mat_aux
            else:
                X_test = np.concatenate((X_test, mat_aux), axis=0)
            c3 += 1

In [16]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(432000, 32)
(144000, 32)
(144000, 32)


In [ ]:
def generar_test(estr):
    with open('EventosAudio.txt', 'r') as f:
        y_data=[]
        n = 0
        for linea in f:
            if n != 0:
                # 0 para audio/no audio, 1 para anger/joy/neutral, 2 para anger/joy/neutral/no sonido
                cleaned_string = re.sub(r'[^a-zA-Z]', '', linea)
                if estr == 0:
                    for _ in range(500):
                        y_data.append('audio')
                    for _ in range(500):
                        y_data.append('no_audio')
                elif estr == 1:
                    for _ in range(1000):
                        y_data.append(cleaned_string)
                #elif estr == 2:
                #    for _ in range(500):
                #        y_data.append(cleaned_string)
                else:
                    for _ in range(500):
                        y_data.append(cleaned_string)
                    for _ in range(500):
                        y_data.append('no_audio')
            n += 1     
    return y_data*int(X_train.shape[0]/96000), y_data*int(X_valid.shape[0]/96000), y_data*int(X_test.shape[0]/96000)

In [ ]:
y_train, y_valid, y_test = generar_test(1)

print(len(y_train))
print(len(y_valid))
print(len(y_test))

conteo = {}
for elemento in y_train:
    conteo[elemento] = conteo.get(elemento, 0) + 1
print(conteo)

432000
144000
144000
{'anger': 144000, 'joy': 144000, 'neutral': 144000}


In [19]:
def apply_sliding_window(X, y, window_size, step_size):
    y_res = []
    X_res = []
    posicion = 0
    stride = window_size - step_size
    total = len(y)
    
    while (posicion + window_size < total):
        y_aux = []
        X_aux = []
        
        for _ in range(window_size):
            y_aux.append(y[posicion])
            X_aux.append(X[posicion])
            posicion += 1
        
        valor_mas_comun = Counter(y_aux).most_common(1)
        y_res.append(valor_mas_comun[0][0])

        X_res.append(np.asarray(X_aux))

        posicion -= stride    
    
    return np.asarray(X_res), np.asarray(y_res)

In [20]:
X_train, y_train = apply_sliding_window(X_train, y_train, window_size=100, step_size=50)
X_valid, y_valid = apply_sliding_window(X_valid, y_valid, window_size=100, step_size=50)
X_test, y_test = apply_sliding_window(X_test, y_test, window_size=100, step_size=50)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(8638, 100, 32)
(2878, 100, 32)
(2878, 100, 32)


In [21]:
print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(8638,)
(2878,)
(2878,)


In [22]:
lista_final = [X_train, X_valid, X_test, y_train, y_valid, y_test]

with open('dataset_X_y_sliding_window.npy', 'wb') as f:
    np.save(f, np.array(lista_final, dtype=object))

In [7]:
dir = './registros/Tabla/Correctos'

directory = os.fsencode(dir)

X_train = np.array([])
X_valid = np.array([])
X_test = np.array([])
y_train = []
y_valid = []
y_test = []

c=0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".mat"):
        mat_aux = scipy.io.loadmat(dir + '/' + filename)['matriz_resultante'].T

        if len(mat_aux) == 96000:
            val3 = len(mat_aux)
            val1 = int(val3*0.6)
            val2 = int(val3*0.6 + val3*0.2)

            if c == 0:
                X_train = mat_aux[0:val1]
                X_valid = mat_aux[val1:val2]
                X_test = mat_aux[val2:val3]
            else:
                X_train = np.concatenate((X_train, mat_aux[0:val1]), axis=0)
                X_valid = np.concatenate((X_valid, mat_aux[val1:val2]), axis=0)
                X_test = np.concatenate((X_test, mat_aux[val2:val3]), axis=0)
            c += 1

def generar_test(estr):
    with open('EventosAudio.txt', 'r') as f:
        y_data=[]
        n = 0
        for linea in f:
            if n != 0:
                # 0 para audio/no audio, 1 para anger/joy/neutral, 2 para anger/joy/neutral/no sonido
                cleaned_string = re.sub(r'[^a-zA-Z]', '', linea)
                if estr == 0:
                    for _ in range(500):
                        y_data.append('audio')
                    for _ in range(500):
                        y_data.append('no_audio')
                elif estr == 1:
                    for _ in range(1000):
                        y_data.append(cleaned_string)
                else:
                    for _ in range(500):
                        y_data.append(cleaned_string)
                    for _ in range(500):
                        y_data.append('no_audio')
            n += 1    

    val3 = len(mat_aux)
    val1 = int(val3*0.6)
    val2 = int(val3*0.6 + val3*0.2)

    return y_data[0:val1]*c, y_data[val1:val2]*c, y_data[val2:val3]*c

In [8]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(864000, 32)
(288000, 32)
(288000, 32)


In [9]:
y_train, y_valid, y_test = generar_test(1)

In [10]:
print(len(y_train))
print(len(y_valid))
print(len(y_test))

846000
282000
282000


In [11]:
X_train, y_train = apply_sliding_window(X_train, y_train, window_size=100, step_size=8)
X_valid, y_valid = apply_sliding_window(X_valid, y_valid, window_size=100, step_size=8)
X_test, y_test = apply_sliding_window(X_test, y_test, window_size=100, step_size=8)

print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(105738,)
(35238,)
(35238,)


In [12]:
lista_final = [X_train, X_valid, X_test, y_train, y_valid, y_test]

with open('dataset_X_y_sliding_window.npy', 'wb') as f:
    np.save(f, np.array(lista_final, dtype=object))